### Alumno: Juan Daniel Kao Pech

In [4]:
import networkx
import os
import xml.etree.ElementTree as ET
import spacy
import networkx as nx
import matplotlib.pyplot as plt
import nltk
import string

In [7]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 3.3 MB/s eta 0:00:04
     ---- ----------------------------------- 1.6/12.8 MB 3.5 MB/s eta 0:00:04
     -------- ------------------------------- 2.6/12.8 MB 3.8 MB/s eta 0:00:03
     ---------- ----------------------------- 3.4/12.8 MB 3.7 MB/s eta 0:00:03
     ------------ --------------------------- 3.9/12.8 MB 3.6 MB/s eta 0:00:03
     ------------- -------------------------- 4.2/12.8 MB 3.2 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 3.0 MB/s eta 0:00:03
     -------------- ------------------------- 4.7/12.8 MB 2.7 MB/s eta 0:00:03
     ---------------- ----------------------- 5.2/12.8 MB 2.6 MB/s eta 0:00:03
     ------------------- -------------------- 6.3/12.8 MB 2.8 MB/s eta 0:00:03
     ---------------------- ----------------- 7.1/12.8 MB 2.9 MB/s

In [10]:
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
directorio = "../Textos de Prueba/"

In [11]:
def lee_xml_eat():
    lista_grafos = []
    G_freq=nx.Graph()
    G_asoc= nx.Graph()
    conjunto_nodos = set()
    tree = ET.parse(directorio+'eat-stimulus-response.xml')
    root = tree.getroot()
    todos = 8211
    i=0
    nodos={}
    total = 0
    j = 0
    for child in root:
        estimulo = child.get('word')
        total =child.get('all')
        estimulo = estimulo.lower()
        estimulo = estimulo.replace(" ","_")
        for response in child.iter('response'):
            frecuencia = response.get('n')
            asociacion = response.get('r')
            respuesta = response.get('word')
            respuesta = respuesta.lower()
            respuesta = respuesta.replace(" ","_")
            if respuesta not in conjunto_nodos:
                nodos[respuesta] = j
                G_freq.add_node(respuesta)
                G_asoc.add_node(respuesta)
                conjunto_nodos.add(respuesta)
                j+=1
            G_freq.add_edge(estimulo,respuesta,weight=float(1/int(frecuencia)))
            G_asoc.add_edge(estimulo,respuesta,weight=float(1-float(asociacion)))
    lista_grafos.append(G_freq)
    lista_grafos.append(G_asoc)
    print("Grafos creados")
    return lista_grafos

In [12]:
grafos = lee_xml_eat()

Grafos creados


In [15]:
g1 = grafos[0]
g2 = grafos[1]

In [20]:
def limpia_lematiza(cadena):
  limpiado = ""
  palabras_funcionales = nltk.corpus.stopwords.words('english')
  for c in string.punctuation:
    cadena = cadena.replace(c, "")
  cadena = cadena.strip()
  for palabra in cadena.split():
    if palabra not in palabras_funcionales:
      doc = nlp(palabra)
      limpiado += doc[0].lemma_ + " "
  return limpiado

In [21]:
limpia_lematiza("an entirely new concept!")

'entirely new concept '

In [22]:
def conceptos(lista,subconjunto):
  datos = []
  if len(lista) <= 100:
    tope = len(lista)
  else:
    tope = 100
  for x in range(0,tope):
    if str(lista[x][0]) not in subconjunto and float(lista[x][1])>0:
      datos.append(str(lista[x][0]))
  return datos

In [ ]:
def diccionario_nap(definicion):
  grafos = lee_xml_eat()
  grafo_frec = grafos[0]
  grafo_asociacion = grafos[1]
  texto = limpia_lematiza(definicion)
  texto = texto.lower()
  tokens = texto.split(" ")
  subconjunto_lemas = []
  for token in tokens:
    if token in grafo_frec.nodes() and token!= "":
      subconjunto_lemas.append(token)
  if len(subconjunto_lemas) > 0:
    resultado_asociacion = nx.betweenness_centrality_subset(grafo_asociacion,subconjunto_lemas,subconjunto_lemas,weight="weight",normalized=True)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x: x[1], reverse=True)[0:99]
    print("Asociación")

    print(conceptos(encontrado,subconjunto_lemas))
    resultado_asociacion = nx.betweenness_centrality_subset(grafo_frec,subconjunto_lemas,subconjunto_lemas,weight="weight",normalized=True)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x: x[1], reverse=True)[0:99]
    print("Frecuencia")
    print(conceptos(encontrado,subconjunto_lemas))
  else:
    print("Definición muy corta, favor de agregar más datos")
  return


In [26]:
diccionario_nap("color that looks like the sky on a clear day")

Grafos creados
Asociación
['blue', 'television', 'now', 'dream', 'bright', 'prefer', 'night', 'regard', 'blues', 'prejudice', 'crystals', 'gazers', 'out', 'another']
Frecuencia
['blue', 'love', 'out', 'see', 'red', 'star', 'cloudy', 'transparent', 'caress', 'kiss', 'lips', 'gazers', 'evening', 'kind', 'night', 'sort', 'appear', "mother's", 'compassionate', 'seem']


In [29]:
diccionario_nap("creamy white tuber that is used in many dishes")

Grafos creados
Asociación
['milk', 'soup', 'potato', 'nodules', 'women', 'them', 'often', 'rabbits', 'pints', 'blight', 'nothing', 'sheep', 'energy', 'instead', 'crop', 'bulb', 'fangs', 'consume', 'wives', 'intestines']
Frecuencia
['up', 'milk', 'potato', 'food', 'good', 'black', 'much', 'mash', 'very', 'beer', 'pint', 'down', 'cat', 'all', 'fish', 'empty', 'money', 'gill', 'broken', 'fill', 'tie', 'fed', 'a_lot', 'flower', 'smashed', 'chips', 'collar', 'most', 'lot', 'vacant', 'pints', 'lily', 'bran', 'vase', 'sausages', 'saucepan', 'plague', 'porcelain', 'dried', 'kitty', 'panther', 'wholesome', 'blight', 'guiness']


In [30]:
diccionario_nap("animal known as the king of the jungle")

Grafos creados
Asociación
['book', 'expert', 'dictionary', 'kingdom', 'gnu', 'rhino', 'lion', 'warrior']
Frecuencia
['book', 'see', 'recognize', 'bird', 'caged', 'edition', 'paper', 'dog', 'board', 'queen', 'story', 'red', 'eye', 'notice', 'look', 'fairy', 'hawk', 'penguin', 'lear', 'tale', 'emery', 'crimson', 'setter', 'quadruped']
